## **[불과 나머지]**<hr>

### [1] 모듈 로딩 <hr>

In [11]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.nn as nn
import torch
import torch.optim as optim

### [2] [데이터 불러오기] <hr>

In [12]:
# 데이터셋 경로
train_dir = './train_data/' 

### [3] 데이터 전처리 & 증강 <hr>

In [13]:

# 학습 데이터에 적용할 transform
train_transform = transforms.Compose([
    transforms.Resize((128, 128)),  # 크기 통일
    transforms.RandomHorizontalFlip(),  # 50% 확률로 좌우 반전
    transforms.RandomRotation(10),     # 최대 10도 회전
    transforms.ToTensor(),             # Tensor 변환
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # RGB 정규화
])

# 검증/테스트 데이터는 데이터 증강 없이
test_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


### [4]  데이터 로더 만들기 <hr>

In [14]:
# Dataset 생성
train_dataset = ImageFolder(root=train_dir, transform=train_transform)
valid_dataset = ImageFolder(root=train_dir, transform=test_transform)  # 여기서는 동일하게 사용, 실제로는 따로 구분 추천

# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

# 데이터셋 클래스 확인
print(f'클래스 : {train_dataset.classes}')
print(f'클래스 라벨 : {train_dataset.class_to_idx}')
print(f'데이터 개수 : {len(train_dataset)}')


클래스 : ['fire', 'others']
클래스 라벨 : {'fire': 0, 'others': 1}
데이터 개수 : 485


### [5] CNN 모델 설계 <hr>

In [15]:

class FireClassifier(nn.Module):
    def __init__(self):
        super(FireClassifier, self).__init__()
        # 특징 추출부 (Convolutional Layers)
        self.conv = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),  # 입력채널 3(RGB), 출력채널 16, 커널 3x3
            nn.ReLU(),
            nn.MaxPool2d(2, 2),              # 크기 절반으로 줄이기
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        # 분류기 (Fully Connected Layers)
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 32 * 32, 128),    # 이미지 크기 128 → 64 → 32 (풀링 2번)
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()                    # Binary Classification
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x


### [6] 학습 준비

In [16]:

# 디바이스 설정 (GPU 사용 가능하면 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

# 모델 인스턴스 생성 및 디바이스 이동
model = FireClassifier().to(device)

# 손실 함수 (Binary Cross Entropy Loss)
criterion = nn.BCELoss()

# 최적화 기법 (Adam)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 관련 설정
EPOCHS = 10
BATCH_SIZE = 32  # 이전에 DataLoader에서 설정한 것과 동일


Using cpu device


### [7]  학습 루프 구현 <hr>

In [17]:
for epoch in range(EPOCHS):
    model.train()  # 학습 모드
    running_loss = 0.0
    correct = 0
    total = 0

    # ---------- 학습 (Training) ----------
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)  # [batch_size, 1] 형태로 변환

        # 1. 기울기 초기화
        optimizer.zero_grad()

        # 2. Forward
        outputs = model(images)

        # 3. Loss 계산
        loss = criterion(outputs, labels)

        # 4. Backward (역전파)
        loss.backward()

        # 5. 가중치 업데이트
        optimizer.step()

        # 통계 기록
        running_loss += loss.item()
        predicted = (outputs >= 0.5).float()  # 0.5 기준으로 이진 분류
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    train_loss = running_loss / len(train_loader)

    # ---------- 검증 (Validation) ----------
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            predicted = (outputs >= 0.5).float()
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)

    val_acc = val_correct / val_total
    val_loss /= len(valid_loader)

    print(f'Epoch [{epoch+1}/{EPOCHS}] '
          f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} '
          f'| Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')


Epoch [1/10] Train Loss: 0.4239, Train Acc: 0.7959 | Val Loss: 0.2808, Val Acc: 0.8866
Epoch [2/10] Train Loss: 0.2641, Train Acc: 0.8722 | Val Loss: 0.2237, Val Acc: 0.9052
Epoch [3/10] Train Loss: 0.1948, Train Acc: 0.9237 | Val Loss: 0.1550, Val Acc: 0.9608
Epoch [4/10] Train Loss: 0.1282, Train Acc: 0.9608 | Val Loss: 0.1193, Val Acc: 0.9608
Epoch [5/10] Train Loss: 0.1197, Train Acc: 0.9588 | Val Loss: 0.0983, Val Acc: 0.9773
Epoch [6/10] Train Loss: 0.0864, Train Acc: 0.9753 | Val Loss: 0.0669, Val Acc: 0.9835
Epoch [7/10] Train Loss: 0.0591, Train Acc: 0.9794 | Val Loss: 0.0611, Val Acc: 0.9835
Epoch [8/10] Train Loss: 0.0846, Train Acc: 0.9753 | Val Loss: 0.0712, Val Acc: 0.9835
Epoch [9/10] Train Loss: 0.1012, Train Acc: 0.9773 | Val Loss: 0.0417, Val Acc: 0.9897
Epoch [10/10] Train Loss: 0.1335, Train Acc: 0.9361 | Val Loss: 0.0854, Val Acc: 0.9588


### [8] 테스트 데이터 평가 + 모델 저장 <hr>

In [18]:
# ---------------------- 테스트 데이터 평가 ----------------------
model.eval()  # 평가 모드
test_correct = 0
test_total = 0

with torch.no_grad():
    for images, labels in valid_loader:  # 여기선 valid_loader를 사용했지만, 실제로는 test_loader 따로 준비 권장
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        outputs = model(images)
        predicted = (outputs >= 0.5).float()
        test_correct += (predicted == labels).sum().item()
        test_total += labels.size(0)

test_acc = test_correct / test_total
print(f"테스트 데이터 정확도 : {test_acc:.4f}")

# ---------------------- 모델 저장 ----------------------
torch.save(model.state_dict(), "fire_classifier_model.pt")
print("모델이 저장되었습니다.")


테스트 데이터 정확도 : 0.9588
모델이 저장되었습니다.


In [19]:
torch.save(model.state_dict(), '../딥러닝/deeproject/model/fire_classifier_model.pt')

In [20]:
import os
os.makedirs('./model', exist_ok=True)

In [21]:
torch.save(model.state_dict(), '../딥러닝/deeproject/model/fire_classifier_model.pt')

In [22]:
# 모델 정의
model = nn.Sequential(
    nn.Conv2d(3, 16, 3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    nn.Conv2d(16, 32, 3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    nn.Flatten(),
    nn.Linear(32 * 37 * 37, 64),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(64, 2)
)

# 저장
import os
os.makedirs('./model', exist_ok=True)
torch.save(model.state_dict(), './model/fire_classifier_model.pt')
